In [17]:
from pymongo import MongoClient
import pandas as pd
import time
import geopandas as gdp
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
from getpass import getpass


In [18]:
client = MongoClient("localhost:27017")

db = client["ironhack"]


In [19]:
db.list_collection_names()

['nyrest', 'Crunchbase', 'restaurants', 'nyneigh']

In [20]:
c = db.get_collection("Crunchbase")

In [22]:
token = getpass()

In [23]:
company_category = set()

In [45]:
# category1 = { "category_code" : {"$exists": true, "$ne": "null"}}
tech_name_regex = {"name":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_category_regex ={"category_code":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_description_regex = {"description":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}
tech_tags_regex = {"tag_list":{"$regex":"^(tech|frontend|backend|web develop|ux develop|ui develop|gaming|gamer)", "$options":"i"}}

tech_cat_or_descr_or_tags = {"$or": [tech_name_regex, tech_category_regex, tech_description_regex, tech_tags_regex]}

projection_name_category_tags = {"_id" : 0, 
                                "name" : 1, 
                                "category_code": 1, 
                                "tag_list": 1,
                                "total_money_raised": 1,
                                "offices.country_code" : 1, 
                                "offices.city" : 1, 
                                "offices.zip_code" : 1, 
                                "offices.latitude" : 1, 
                                "offices.longitude" : 1}

tech_companies_list = list(c.find(tech_cat_or_descr_or_tags, projection_name_category_tags).sort("offices.city", 1))

In [47]:
df_tech_general = pd.DataFrame(tech_companies_list)

In [88]:
df_tech_general.sample(5)

,name,category_code,tag_list,total_money_raised,offices
107,Holdsworth Associates,public_relations,"pr, public-relations, communications, marketin...",$0,"[{'zip_code': 'CB24 4QG', 'city': 'Cambridge',..."
257,TechTeam Global,enterprise,,$0,"[{'zip_code': '48033', 'city': 'Southfield', '..."
258,FileFront,web,,$0,"[{'zip_code': '77388', 'city': 'Spring', 'coun..."
283,GoPlanit,web,"techcrunch50, tc50, travel, trip-planner, mobi...",$500k,"[{'zip_code': '94111', 'city': 'san francisco'..."
150,Quant the News,web,"techcrunch50, tc50, finance, subscription-serv...",$150k,"[{'zip_code': '90067', 'city': 'Los Angeles', ..."


In [89]:
design_name_regex = {"name":{"$regex":"design", "$options":"i"}}
design_category_regex ={"category_code":{"$regex":"design", "$options":"i"}}
design_description_regex = {"description":{"$regex":"design", "$options":"i"}}
design_tags_regex = {"tag_list":{"$regex":"design", "$options":"i"}}

design_cat_or_descr_or_tags = {"$or": [design_name_regex, design_category_regex, design_description_regex, design_tags_regex]}

design_projection_name_category_tags = {"_id" : 0, 
                                        "name" : 1, 
                                        "category_code": 1, 
                                        "tag_list": 1,
                                        "offices.country_code" : 1, 
                                        "offices.city" : 1, 
                                        "offices.zip_code" : 1, 
                                        "offices.latitude" : 1, 
                                        "offices.longitude" : 1}

design_companies_list = list(c.find(design_cat_or_descr_or_tags, design_projection_name_category_tags).sort("offices.city", 1))

In [90]:
len(design_companies_list)

779

In [91]:
df_design = pd.DataFrame(design_companies_list)

In [96]:
df_design.sample(5)

,name,category_code,tag_list,offices
471,FUSE,web,"web-design, 3d, 3d-animation, microsite, widge...","[{'zip_code': 'W1D 7AZ', 'city': 'London', 'co..."
180,Web:Studio,network_hosting,"web-design, web-hosting, domain-names, graphic...","[{'zip_code': 'Co Louth', 'city': 'Ardee', 'co..."
5,modula,search,"web-design, software-development, graphic-desi...",[]
640,Phu Concepts,advertising,"phu-concepts-inc, web-design, viet, latrell, v...","[{'zip_code': '14605', 'city': 'Rochester', 'c..."
652,IncSpring,public_relations,"logo, logo-design, brand, branding, marketing,...","[{'zip_code': '78216', 'city': 'San Antonio', ..."
